In [ ]:
 current_dir = "gdrive/My Drive/Colab Notebooks/Thesis/gold_standard"


import os
import glob
from google.colab import drive
import pandas as pd
import re
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import re
from sklearn.linear_model import SGDRegressor
from sklearn.svm import SVR
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline


from sklearn.linear_model import (
    LinearRegression,
    Ridge,
    Lasso,
    ElasticNet,
    RidgeCV,
    LassoCV,
    ElasticNetCV,
)


# Mount Google Drive
drive.mount('/content/gdrive')



Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
os.chdir(current_dir)

FileNotFoundError: ignored

In [ ]:
import pandas as pd
copy = pd.read_pickle("pos_dataframe.pkl")

#Experiments
##Train: handbook
##Validate: doc
##Test: cbc & article

In [ ]:
columns_to_keep  = ['ADJ', 'ADP', 'ADV', 'AUX', 'DET', 'INTJ', 'NOUN', 'NUM', 'PART', 'PRON', 'SCONJ', 'VERB', 'group', 'name', "score"]

In [ ]:
df = copy[columns_to_keep]
# Split the DataFrame into 'train' and 'validate' based on the 'group' column
train = df[df['group'] == 'handbook']
validate = df[df['group'] == 'doc']
test = df[~df['group'].isin(['handbook', 'doc'])]

In [ ]:
print(df)

In [ ]:
models = [
    LinearRegression(),
    Ridge(),
    Lasso(),
    ElasticNet(),
    SVR(),
]
features = ['ADJ', 'ADP', 'ADV', 'AUX', 'DET', 'INTJ', 'NOUN', 'NUM', 'PART', 'PRON', 'SCONJ', 'VERB',]

In [ ]:
from itertools import combinations

features = ['ADJ', 'ADP', 'ADV', 'AUX', 'DET', 'INTJ', 'NOUN', 'NUM', 'PART', 'PRON', 'SCONJ', 'VERB']
feature_combos = [list(combo) for combo in combinations(features, 2)]

In [ ]:
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression

def create_and_train_model(train_data, feature_columns, model):
    loo = LeaveOneOut()
    y_preds, y_trues = [], []

    # Create and fit CountVectorizers outside the loop
    vectorizers = {col: CountVectorizer().fit(train_data[col]) for col in feature_columns}

    # Create the ColumnTransformer with fitted CountVectorizers
    transformers = [(col, vectorizers[col], col) for col in feature_columns]
    preprocessor = ColumnTransformer(transformers=transformers, remainder='drop')

    for train_index, test_index in loo.split(train_data):
        X_train, X_test = train_data."gold_psy.pkl"iloc[train_index], train_data.iloc[test_index]
        y_train, y_test = train_data['score'].iloc[train_index], train_data['score'].iloc[test_index]

        # Create a pipeline with preprocessor and model
        pipe = Pipeline([('preprocessor', preprocessor), ('model', model)])
        # Fit the pipeline on the training data
        pipe.fit(X_train[feature_columns], y_train)
        y_pred = pipe.predict(X_test[feature_columns])

        # Extract single prediction and actual value
        y_pred_single = y_pred[0]
        y_preds.append(y_pred)
        y_trues.append(y_test.iloc[0])


    metrics = {
        'R²': r2_score(y_trues, y_preds),
        'MSE': mean_squared_error(y_trues, y_preds),
        'RMSE': np.sqrt(mean_squared_error(y_trues, y_preds)),
        'MAE': mean_absolute_error(y_trues, y_preds)
    }

    return {
        'Trained Pipeline': pipe,
        'Model': str(model),
        'Feature Columns': feature_columns,
        'Evaluation Metrics': metrics
    }


In [ ]:
def validate_trained_pipe(val_data, pipeline):
    print(pipeline)
    # Predict using the pipeline
    y_pred = pipeline.predict(val_data)
    y_true = val_data['score']
    # Calculate evaluation metrics
    metrics = {
        'R²': r2_score(y_true, y_pred),
        'MSE': mean_squared_error(y_true, y_pred),
        'RMSE': np.sqrt(mean_squared_error(y_true, y_pred)),
        'MAE': mean_absolute_error(y_true, y_pred)
    }
    # Print Names, Actual Scores, and Predicted Scores
    for name, actual, predicted in zip(val_data['name'], y_true, y_pred):
        print(f"Name: {name}, Actual Score: {actual}, Predicted Score: {predicted}")
    return metrics






In [ ]:
for feature_combo in feature_combos:
  for model in models:
    pipe_data = create_and_train_model(train, feature_combo, model)
    if pipe_data["Evaluation Metrics"]["R²"] > 0.9:
      validation_data = validate_trained_pipe(validate, pipe_data["Trained Pipeline"])
      print(validation_data)



Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('ADV', CountVectorizer(),
                                                  'ADV'),
                                                 ('AUX', CountVectorizer(),
                                                  'AUX')])),
                ('model', Lasso())])
Name: wayne, Actual Score: 40.0, Predicted Score: 38.86966991622276
Name: carl, Actual Score: 36.0, Predicted Score: 39.51069880255482
{'R²': -0.7003315225689637, 'MSE': 6.801326090275855, 'RMSE': 2.6079352158893547, 'MAE': 2.3205144431660294}
Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('ADV', CountVectorizer(),
                                                  'ADV'),
                                                 ('AUX', CountVectorizer(),
                                                  'AUX')])),
                ('model', ElasticNet())])
Name: wayne, Actual Score: 40.0, Predicted Score: 38.896197591262535
